In [ ]:
from __future__ import division, print_function
import sys, os, glob, time, warnings, gc
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table, vstack, hstack, join
import fitsio

In [ ]:
params = {'legend.fontsize': 'large',
         'axes.labelsize': 'large',
         'axes.titlesize':'large',
         'xtick.labelsize':'large',
         'ytick.labelsize':'large',
         'figure.facecolor':'w'} 
plt.rcParams.update(params)

In [ ]:
# Load DESI Y1 zcatalog
desi = Table(fitsio.read('/global/cfs/cdirs/desi/public/dr1/spectro/redux/iron/zcatalog/zpix-main-dark.fits',
                         columns=['TARGETID', 'COADD_FIBERSTATUS', 'ZWARN', 'DESI_TARGET', 'Z']))
print(len(desi))

# Remove FIBERSTATUS!=0 fibers
mask = desi['COADD_FIBERSTATUS']==0
# Remove ZWARN!=0 fibers
mask &= desi['ZWARN']==0
# Select QSO targets
mask &= (desi['DESI_TARGET'] & 2**2>0)
desi = desi[mask]
print(len(desi))

Load DR16Q cross-matched catalog. __Use *DR9* cross-matched catalog as DR9 is the version used in DESI targeting.__

In [ ]:
ls = Table(fitsio.read('/global/cfs/cdirs/desi/target/analysis/truth/dr9.0/south/matched/ls-dr9.0-DR16Q_v4-match.fits', columns=['OBJID', 'BRICKID', 'RELEASE']))
truth = Table(fitsio.read('/global/cfs/cdirs/desi/target/analysis/truth/dr9.0/south/matched/DR16Q_v4-match.fits', columns=['Z']))

Add DESI TARGETID to the (pre-matched) external catalog

In [ ]:
from desitarget import targets
ls['TARGETID'] = targets.encode_targetid(ls['OBJID'], ls['BRICKID'], ls['RELEASE'])

# Match to redrock catalog
_, idx_desi, idx_truth = np.intersect1d(desi['TARGETID'], ls['TARGETID'], return_indices=True)
print('Found {} matched objects'.format(len(idx_desi)))

In [ ]:
sys.path.append(os.path.expanduser('~/git/desi-examples/sv'))
from util import compare_redshifts  # https://github.com/rongpu/desi-examples/blob/master/sv/util.py

outlier_threshold = 0.01
compare_redshifts(truth['Z'][idx_truth], desi['Z'][idx_desi],
                markersize=0.5, alpha=1, zmin=-0.05, zmax=4.5, outlier_threshold=outlier_threshold, dz_range=0.01,
                xlabel='$z_{\mathrm{SDSS}}$', ylabel='$z_{\mathrm{DESI}}$', ylabel2='$\\Delta z/(1+z_{\\mathrm{SDSS}})$')